In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

froot = './data/k562_chr16'

df = pd.read_csv(froot + ".csv")

column_names = np.array(df.columns)
feature_names = column_names[6:-6]
num_features = len(feature_names)
print(feature_names)
num_samples = df.shape[0]

#Y_ji is a list of samples containing lists of their feature values
    # [   
    #   sample_1: [feat_1, feat_2,...,feat_n],
    #   sample_2: [feat_1, feat_2,...,feat_n],
    # ]

Y_ji = df.iloc[:, 6:-6].values

# process GLM simulated elongation rates
Z_ji = df['zeta'].values

model = nn.Linear(num_features, 1, bias=False, dtype=torch.float32)
model.weight.data[0] = torch.tensor([-0.02, -0.9, -0.02, 0.03, -0.06, -0.03, -0.05, -0.02, -0.03, -0.06, 0.01, -0.18], dtype=torch.float32)

model.eval()

weights = model.weight.data.cpu().numpy()
combined = ', '.join([f'"{s}": {f}' for s, f in zip(feature_names, weights[0])])
print(combined)

['ctcf' 'h3k36me3' 'h3k4me1' 'h3k79me2' 'h3k9me1' 'h3k9me3' 'h4k20me1'
 'sj5' 'sj3' 'dms' 'rpts' 'wgbs']
['A' 'T' 'G' 'C']
"ctcf": -0.019999999552965164, "h3k36me3": -0.8999999761581421, "h3k4me1": -0.019999999552965164, "h3k79me2": 0.029999999329447746, "h3k9me1": -0.05999999865889549, "h3k9me3": -0.029999999329447746, "h4k20me1": -0.05000000074505806, "sj5": -0.019999999552965164, "sj3": -0.029999999329447746, "dms": -0.05999999865889549, "rpts": 0.009999999776482582, "wgbs": -0.18000000715255737


In [9]:
with torch.no_grad():
    y_inputs = Y_ji
    rho_ji = model(torch.tensor(y_inputs, dtype=torch.float32))

    matmul_zeta = torch.exp(torch.matmul(torch.tensor(Y_ji, dtype=torch.float32), model.weight.data[0]))
    
    # convert log(Z) outputs to Z
    neural_net_zeta = torch.exp(rho_ji.squeeze())
    
    print("Neural Network Zeta:")
    print(neural_net_zeta.numpy())
    print("\n")

    print("Matrix Multiplication Zeta: Z_ji = e^(k * Y_ji)")
    print(matmul_zeta.numpy())
    print("\n")
    
    print("Ling Zeta:")
    print(Z_ji)
    

Neural Network Zeta:
[1.527791  1.3849593 1.5305036 ... 1.1874436 1.1889294 1.1904222]


Matrix Multiplication Zeta: Z_ji = e^(k * Y_ji)
[1.527791  1.3849593 1.5305036 ... 1.1874436 1.1889294 1.1904222]


Ling Zeta:
[1.08215648 0.89331623 1.08149273 ... 0.90863945 0.90872591 0.90880907]
